In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import gc
import glob
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
sys.path.append(f"../py/")
import MS_utils
import utils, ml_utils
from utils import logger_func
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

pd.set_option("max_columns", 200)
pd.set_option("max_rows", 400)

# Columns
key, target, ignore_list = MS_utils.get_basic_var()
ignore_list = [key, target]

# Basic Args
seed = 1208
set_type = 'all'

2019-03-08 23:26:10,081 utils 346 [INFO]    [logger_func] start 


In [3]:
target_list = [
    'f000_SmartScreen'
    ,'f000_AvSigVersion'
    ,'f000_EngineVersion'
    ,'f000_AppVersion'
    ,'f000_Census_OSVersion'
    ,'f000_AVProductStatesIdentifier'
    ,'f000_CountryIdentifier'
    ,'f000_AVProductsInstalled'
    ]

agg_list = [
    'f000_SmartScreen'
    ,'f000_AVProductStatesIdentifier'
    ,'f000_CountryIdentifier'
    ,'f000_Census_ActivationChannel'
    ,'f000_Census_OSInstallTypeName'
    ,'f000_Census_TotalPhysicalRAM'
    ,'f000_Census_PrimaryDiskTotalCapacity'
    ,'f000_AVProductsInstalled'
    ,'f000_Census_GenuineStateName'
    ,'f000_Wdft_RegionIdentifier'
]

no_im_list = [
    'f000_Census_TotalPhysicalRAM'
    ,'f000_Census_PrimaryDiskTotalCapacity'
]

encode_list = [
    'f009_AvSigVersion-Major'
    ,'f009_Engine-Major'
    ,'f009_AppVersion-1'
    ,'f009_AppVersion-3'
    ,'f009_OSVersion-Major'
    ,'f009_OSVersion-Sub'
]

In [4]:
clean_list = []
feat_path_list = glob.glob('../features/1_first_valid/*.gz')
feat_path_list = glob.glob('../features/viz/*.gz')
# feat_path_list = glob.glob('../features/4_winner/f013*.gz')
# feat_path_list = ml_utils.get_feat_path_list(clean_list, file_key='f000')
p_list = utils.parallel_load_data(path_list=feat_path_list)
tmp_feat = pd.concat(p_list, axis=1)
base = utils.read_df_pkl('../input/base_Av*')[[key, target, 'country_group']]
df_feat = pd.concat([base, tmp_feat], axis=1)
df_feat.head()

100%|██████████| 3/3 [00:01<00:00,  2.96it/s]


,MachineIdentifier,HasDetections,country_group,f000_EngineVersion,f000_Census_OSVersion,f009_AppVersion-1,f009_OSVersion-Major,f000_SmartScreen,f009_Engine-Major,f009_AvSigVersion-Major,f000_AvSigVersion,f012_org-Engine-Major,f000_AppVersion
0,0,0.0,4,1.1.15100.1,10.0.17134.165,18.0,17134.0,NaN,15100.0,127.0,1.273.1735.0,0.0,4.18.1807.18075
1,1,0.0,6,1.1.14600.4,10.0.17134.1,13.0,17134.0,NaN,14600.0,126.0,1.263.48.0,14600.0,4.13.17134.1
2,2,0.0,5,1.1.15100.1,10.0.17134.165,18.0,17134.0,RequireAdmin,15100.0,127.0,1.273.1341.0,0.0,4.18.1807.18075
3,3,1.0,2,1.1.15100.1,10.0.17134.228,18.0,17134.0,ExistsNotSet,15100.0,127.0,1.273.1527.0,0.0,4.18.1807.18075
4,4,1.0,2,1.1.15100.1,10.0.17134.191,18.0,17134.0,RequireAdmin,15100.0,127.0,1.273.1379.0,0.0,4.18.1807.18075


In [ ]:
#========================================================================
# Cleansing
#========================================================================

feat_key = 'f00'
# feat_key = 'f012'

# Impute
feat_cols = [col for col in df_feat.columns if col.count(feat_key)]

for col in feat_cols:
    if str(df_feat[col].dtype).count('float') or str(df_feat[col].dtype).count('int'):
        df_feat.loc[df_feat[col].isnull(), col] = -1
    elif str(df_feat[col].dtype).count('cate'):
        df_feat.loc[df_feat[col].isnull(), col] = df_feat[col].mode()
    else:
        df_feat.loc[df_feat[col].isnull(), col] = 'N'

# Clean Imbalance
train = df_feat[df_feat[target].isnull()]
test = df_feat[~df_feat[target].isnull()]
for col in feat_cols:
    if col in no_im_list:
        continue
    
    cnt_train = train.groupby(col).size()
    cnt_test = test.groupby(col).size()
    
    df_cnt = pd.concat([cnt_train, cnt_test], axis=1).fillna(0) + 1
    df_cnt.columns = ['train', 'test']
    
    df_cnt['total'] = df_cnt['train'] + df_cnt['test']
    
    im_train = (df_cnt['train'] / df_cnt['total'])<0.2
    im_cat_list = list(set(df_cnt.loc[im_train].index.tolist() + df_cnt[df_cnt['train']<1000].index.tolist() + df_cnt[df_cnt['test']<1000].index.tolist()))
    
    if len(im_cat_list):
        print(col)
        print(im_cat_list)
        if (str(df_feat[col].dtype).count('float') or str(df_feat[col].dtype).count('int')) and 0 not in im_cat_list:
            df_feat.loc[df_feat[col].isin(im_cat_list), col] = 0
        else:
            df_feat.loc[df_feat[col].isin(im_cat_list), col] = df_feat[col].mode()
            
    im_test = (df_cnt['test'] / df_cnt['total'])<0.2
    im_cat_list = list(set(df_cnt.loc[im_test].index.tolist() + df_cnt[df_cnt['test']<1000].index.tolist() + df_cnt[df_cnt['test']<1000].index.tolist()))
    
    if len(im_cat_list):
        if str(df_feat[col].dtype).count('float') or str(df_feat[col].dtype).count('int'):
            df_feat.loc[df_feat[col].isin(im_cat_list), col] = 0
        else:
            df_feat.loc[df_feat[col].isin(im_cat_list), col] = df_feat[col].mode()
df_feat.head()

In [35]:
#========================================================================
# Saving
#========================================================================
prefix = 'f012'
prefix = 'f013'
feat_key = 'f012'
feat_list = [col for col in df_feat.columns if col.count(feat_key)]
for col in feat_list:
    feature = df_feat[col].values
    col = col.replace(feat_key, prefix)
    utils.to_pkl_gzip(obj=feature, path=f'../features/1_first_valid/{col}')

In [130]:
sm0_col = 'f000_SmartScreen'
sm8_col = 'f008_SmartScreen'
vi0_col = 'f000_AvSigVersion-Major'
vi9_col = 'f009_AvSigVersion-Major'
en0_col = 'f000_EngineVersion'
en9_col = 'f009_Engine-Major'
en12_col = 'f012_org-Engine-Major'
app0_col = 'f000_AppVersion'
app91_col = 'f009_AppVersion-1'
app93_col = 'f009_AppVersion-3'
os0_col = 'f000_Census_OSVersion'
os9m_col = 'f009_OSVersion-Major'
os9s_col = 'f009_OSVersion-Sub'

en13_col = 'f013_org-Engine-Major'
os13m_col = 'f013_OSVersion-Major'
vi13_col = 'f013_AvSigVersion-Major'

In [ ]:
#========================================================================
# 重要変数を個別クレンジングする 
# df_feat[en0_col] = utils.read_pkl_gzip('../features/viz/f000_EngineVersion.gz')
# df_feat[vi0_col] = utils.read_pkl_gzip('../features/viz/f000_AvSigVersion.gz')
# df_feat[sm8_col] = utils.read_pkl_gzip('../features/no_use/f008_SmartScreen.gz')
# sys.exit()

train, test = ml_utils.split_train_test(df_feat, target)
def comp_train_test(train, test, comp_list):
    cnt_train = train.groupby(comp_list).size()
    cnt_test = test.groupby(comp_list).size()
    
    cnt = pd.concat([cnt_train, cnt_test], axis=1)
    cnt.columns = ['train', 'test']
    cnt['total'] = cnt['train'] + cnt['test']
    cnt['ratio'] = cnt['train'] / cnt['total']
    
    return cnt
# sys.exit()
# comp_list = [en9_col, en12_col, vi9_col]
# comp_list = [en12_col, vi9_col]
# comp_list = [en12_col, app91_col]
# comp_list = [en12_col, os9m_col]
# comp_list = [os13m_col, vi13_col]
comp_list = [en12_col, os0_col, sm0_col]
cnt = comp_train_test(train, test, comp_list)
cnt.sort_values(by='total', ascending=False)
#========================================================================

In [ ]:
# cnt.loc[15100, :]

In [52]:
from tqdm import tqdm
down_os_list = train[train[en9_col]==15100][os0_col].unique().tolist()
df_en9_15100 = train[train[en9_col]==15100]
down_id_list = []
for os in tqdm(down_os_list):
    down_id_list += df_en9_15100.loc[df_en9_15100[os0_col]==os, key].sample(frac=0.90).values.tolist()
    
df_down = train[~train[key].isin(down_id_list)]
print(len(down_id_list))
print(df_down.shape)

100%|██████████| 384/384 [01:37<00:00,  3.88it/s]


3308345
(5613138, 13)


In [4]:
from sklearn.model_selection import KFold
base = utils.read_df_pkl('../input/base_gr*')
# base['down_flg'] = 0
# base.loc[(base[key].isin(down_id_list)) & (~base[target].isnull()), 'down_flg'] = 1
# base['down_flg'].value_counts()
# utils.to_df_pkl(df=base, path='../input', fname='base_group')
tmp = base[base['down_flg']==0][~base[target].isnull()].reset_index()
fold_n = 5
fold_seed = 1208
cv = KFold(n_splits=fold_n, shuffle=True, random_state=fold_seed)
kfold = list(cv.split(tmp, tmp[target]))
# kfold = MS_utils.get_kfold(base=tmp)

100%|██████████| 3/3 [00:00<00:00,  5.87it/s]


In [8]:
max(set(kfold[0][1].tolist()))

5613131

In [27]:
#========================================================================
# EngineとAvSigのクレンジング
# f009とimbalanceの後処理なので注意
df_feat.loc[(df_feat[en12_col]==0) & (df_feat[vi9_col]<127), vi9_col] = 127
df_feat.loc[(df_feat[en12_col]==12902), vi9_col] = 122
df_feat.loc[(df_feat[en12_col]==13103), vi9_col] = 122
df_feat.loc[(df_feat[en12_col]==13303), vi9_col] = 123
df_feat.loc[(df_feat[en12_col]==13407), vi9_col] = 123
df_feat.loc[(df_feat[en12_col]==13504), vi9_col] = 123
df_feat.loc[(df_feat[en12_col]==13701), vi9_col] = 123
df_feat.loc[(df_feat[en12_col]==13704), vi9_col] = 124
df_feat.loc[(df_feat[en12_col]==14104), vi9_col] = 125
df_feat.loc[(df_feat[en12_col]==14500), vi9_col] = 126
df_feat.loc[(df_feat[en12_col]==14600), vi9_col] = 126
df_feat.loc[(df_feat[en12_col]==14700), vi9_col] = 126
df_feat.loc[(df_feat[en12_col]==14800), vi9_col] = 126
df_feat.loc[(df_feat[en12_col]==14901) & (df_feat[vi9_col]<=122), vi9_col] = 122
df_feat.loc[(df_feat[vi9_col] >127), vi9_col] = 127

df_feat.loc[(df_feat[en12_col]==13202) & (df_feat[vi9_col]==122), vi9_col] = 123
df_feat.loc[(df_feat[en12_col]==13601) & (df_feat[vi9_col]==124), vi9_col] = 123
df_feat.loc[(df_feat[en12_col]==13701) & (df_feat[vi9_col]==124), vi9_col] = 123
df_feat.loc[(df_feat[en12_col]==14104) & (df_feat[vi9_col]==124), vi9_col] = 125
df_feat.loc[(df_feat[en12_col]==14600) & (df_feat[vi9_col]==124), vi9_col] = 126

df_feat.loc[(df_feat[en12_col]==14405), en12_col] = 14500
df_feat.loc[(df_feat[en12_col]==14901) & (df_feat[vi9_col]==127), en12_col] = 15000
df_feat.loc[(df_feat[en12_col]==15200) & (df_feat[vi9_col]<=124), en12_col] = 14901
df_feat.loc[(df_feat[en12_col]==14901) & (df_feat[vi9_col]==123), vi9_col] = 124

df_feat.loc[(df_feat[en12_col]==14202), en12_col] = 14104
df_feat.loc[(df_feat[en12_col]==14306), en12_col] = 14104

#========================================================================

In [28]:
#========================================================================
# f012_org-Engineとf009_AvSigクレンジング後
# f009_OSVersionクレンジング
df_feat.loc[(df_feat[en12_col]==15200) & (df_feat[os9m_col]>=17134), os9m_col] = 17134
df_feat.loc[(df_feat[en12_col]<=15000) & (df_feat[os9m_col]>=17500), os9m_col] = 17134
df_feat.loc[(df_feat[en12_col]==0) & (df_feat[os9m_col]==17758), os9m_col] = 0
df_feat.loc[(df_feat[en12_col]==12805) & (df_feat[os9m_col]==0), os9m_col] = 14393
df_feat.loc[(df_feat[en12_col]==12902) & (df_feat[os9m_col]==0), os9m_col] = 10586
df_feat.loc[(df_feat[en12_col]==13103) & (df_feat[os9m_col]==0), os9m_col] = 10586
df_feat.loc[(df_feat[en12_col]<=15000) & (df_feat[os9m_col]==0), os9m_col] = 10586
df_feat.loc[(df_feat[os9m_col]==10240), os9m_col] = 10586
df_feat.loc[(df_feat[os9m_col]==10240), os9m_col] = 10586
df_feat.loc[(df_feat[os9m_col]==10240), os9m_col] = 10586
#========================================================================

In [13]:
#========================================================================
# Engine Cleansing後, OS Cleansing前
# Engine-majorに対して最も多いOSversionを取得する
#========================================================================

def major_version_diff(df_feat, key_ver_col, diff_ver_col):
    
    train, test = ml_utils.split_train_test(df_feat, target)
    cnt = comp_train_test(train, test, [key_ver_col, diff_ver_col])
    
    tmp_cnt = cnt.reset_index()
    tmp_cnt = tmp_cnt.groupby(key_ver_col)['total'].max()
    tmp_cnt.name = 'max_total'
    tmp_cnt = cnt.join(tmp_cnt)
    tmp_cnt = tmp_cnt[tmp_cnt['max_total']==tmp_cnt['total']]
    tmp_cnt.reset_index(inplace=True)
    major_ver = tmp_cnt.iloc[:, :2]
    major_ver.rename(columns={diff_ver_col:'major_ver'}, inplace=True)
    
    # 重複が発生してないかチェック
    tmp = major_ver.groupby(key_ver_col).size()
    if len(tmp[tmp>1]):
        display(tmp)
        sys.exit()
    
    df_feat = df_feat.merge(major_ver, how='left', on=key_ver_col)
    df_feat[f'major_ver_diff_{key_ver_col}-{diff_ver_col}'] = df_feat[diff_ver_col] - df_feat['major_ver']
    df_feat.drop('major_ver', axis=1, inplace=True)
    
    return df_feat
    
kuni12_col = 'f012_CountryIdentifier'
city8_col = 'f008_CityIdentifier'
local8_col = 'f008_LocaleEnglishNameIdentifier'
region12_col = 'f012_Wdft_RegionIdentifier'

# df_feat[kuni12_col] = utils.read_pkl_gzip('../features/4_winner/f012_CountryIdentifier.gz')
# df_feat[city8_col] = utils.read_pkl_gzip('../features/4_winner/f008_CityIdentifier.gz')
# df_feat[local8_col] = utils.read_pkl_gzip('../features/4_winner/f008_LocaleEnglishNameIdentifier.gz')
# df_feat[region12_col] = utils.read_pkl_gzip('../features/4_winner/f012_Wdft_RegionIdentifier.gz')


#========================================================================
# Major Diffの取得
#========================================================================
# df_feat = major_version_diff(df_feat, en12_col, os9m_col)
# df_feat = major_version_diff(df_feat, en12_col, app91_col)
# df_feat = major_version_diff(df_feat, en12_col, sm8_col)
# df_feat = major_version_diff(df_feat, app91_col, os9m_col)

# df_feat = major_version_diff(df_feat, kuni12_col, os9m_col)
# df_feat = major_version_diff(df_feat, kuni12_col, en12_col)
# df_feat = major_version_diff(df_feat, kuni12_col, app91_col)
# df_feat = major_version_diff(df_feat, kuni12_col, sm8_col)

# df_feat = major_version_diff(df_feat, city8_col, os9m_col)
# df_feat = major_version_diff(df_feat, city8_col, en12_col)
# df_feat = major_version_diff(df_feat, city8_col, app91_col)
# df_feat = major_version_diff(df_feat, city8_col, sm8_col)

# df_feat = major_version_diff(df_feat, local8_col, os9m_col)
# df_feat = major_version_diff(df_feat, local8_col, en12_col)
# df_feat = major_version_diff(df_feat, local8_col, app91_col)
# df_feat = major_version_diff(df_feat, local8_col, sm8_col)

# df_feat = major_version_diff(df_feat, region12_col, os9m_col)
# df_feat = major_version_diff(df_feat, region12_col, en12_col)
# df_feat = major_version_diff(df_feat, region12_col, app91_col)
# df_feat = major_version_diff(df_feat, region12_col, sm8_col)

# drop

# df_feat.drop(['major_ver_diff_f008_SmartScreen-f009_AppVersion-1'], axis=1, inplace=True)
# df_feat.shape
# country = utils.read_pkl_gzip('../features/4_winner/f012_CountryIdentifier.gz')
# print(country.shape)

#========================================================================
# diff major featureのsave
#========================================================================
major_col = [col for col in df_feat.columns if col.count('major_')]
prefix = 'f013'
# for col in major_col:
# #     if col.count('Engine') and col.count('OSVer'):
#     feature = df_feat[col].values.astype('float32')
#     utils.to_pkl_gzip(obj=feature, path=f'../features/1_first_valid/{prefix}_{col}')
# df_feat.drop(major_col, axis=1, inplace=True)
df_feat.head()

,MachineIdentifier,HasDetections,country_group,f000_EngineVersion,f009_AppVersion-1,f009_OSVersion-Major,f000_SmartScreen,f009_Engine-Major,f009_AvSigVersion-Major,f000_AvSigVersion,f012_org-Engine-Major
0,0,0.0,4,1.1.15200.1,18.0,17134.0,N,0.0,127.0,1.263.48.0,0.0
1,1,0.0,6,1.1.14600.4,13.0,17134.0,N,14600.0,126.0,1.263.48.0,14600.0
2,2,0.0,5,NaN,18.0,17134.0,RequireAdmin,0.0,127.0,NaN,0.0
3,3,1.0,2,NaN,18.0,17134.0,ExistsNotSet,0.0,127.0,NaN,0.0
4,4,1.0,2,NaN,18.0,17134.0,RequireAdmin,0.0,127.0,NaN,0.0


In [14]:
vi9_col = 'f009_AvSigVersion-Major'
en0_col = 'f000_EngineVersion'
en12_col = 'f012_Engine-Major'
df_feat = utils.read_df_pkl('../input/base_Av*')
df_feat[en0_col] = utils.read_pkl_gzip('../features/4_winner/f000_EngineVersion.gz')
df_feat[en12_col] = utils.read_pkl_gzip('../features/no_use/f012_Engine-Major.gz')
df_feat[vi9_col] = utils.read_pkl_gzip('../features/no_use/f009_AvSigVersion-Major.gz')

100%|██████████| 3/3 [00:01<00:00,  2.94it/s]


In [15]:
#========================================================================
# f012_org-Engineとf009_AvSigクレンジング後のダウンサンプリングフラグ作成
dw_flg = 'down_flg'
df_feat['down_flg'] = 0

df_feat.loc[(df_feat[en0_col]==10701) , dw_flg] = 1
df_feat.loc[(df_feat[en0_col]==11202) , dw_flg] = 1
df_feat.loc[(df_feat[en0_col]==12802) , dw_flg] = 1
df_feat.loc[(df_feat[en0_col]==13301) , dw_flg] = 1

df_feat.loc[(df_feat[en12_col]==14800) & (df_feat[vi9_col]<=125), dw_flg] = 1
df_feat.loc[(df_feat[en12_col]==14901) & (df_feat[vi9_col]<=125), dw_flg] = 1
df_feat.loc[(df_feat[en12_col]==15000) & (df_feat[vi9_col]<=125), dw_flg] = 1
df_feat.loc[(df_feat[en12_col]==15100) & (df_feat[vi9_col]<=125), dw_flg] = 1
df_feat.loc[(df_feat[en12_col]==15200) & (df_feat[vi9_col]<=126), dw_flg] = 1

"""
一部をダウンサンプリングする場合は2
"""
df_feat.loc[(df_feat[en12_col]==15100) & (df_feat[vi9_col]<=127), dw_flg] = 2 # 3200000 down sample
#========================================================================

0.0        14961599
15000.0      335514
14600.0      299099
14901.0      290649
14800.0      193916
14104.0      166721
13504.0      120050
14700.0       71667
14500.0       70214
14405.0       55619
14306.0       37335
14202.0       24814
14003.0       21437
13903.0       14859
13303.0       14618
13804.0       14552
13407.0       13678
13601.0       11407
12902.0        8503
14305.0        8210
13701.0        7726
13704.0        7582
13202.0        7460
13103.0        6917
13000.0        6430
12805.0        4160
Name: f012_Engine-Major, dtype: int64

In [31]:
#========================================================================
# Avsigの個別処理
# ;f_feat.loc[df_feat['f012_AvSigVersion-Major']==0, 'f009_AvSigVersion-Major'] = 127.0
df_feat[df_feat[target].isnull()]['f009_AvSigVersion-Major'].value_counts()
#========================================================================
# df_feat

127.0    5894312
126.0     351618
125.0     130539
123.0      69316
124.0      23327
122.0      10417
Name: f009_AvSigVersion-Major, dtype: int64

In [42]:
#========================================================================
# Engineの個別処理
en_col = 'f012_org-Engine-Major'
encode_idx_1 = (df_feat['f009_Engine-Major']==15300) & (df_feat['f012_AvSigVersion-Major']==126)
encode_idx_2 = (df_feat['f009_Engine-Major']==15400) & (df_feat['f012_AvSigVersion-Major']==126)
encode_idx_3 = (df_feat['f009_Engine-Major']==15200) & (df_feat['f012_AvSigVersion-Major']==126)
encode_idx_4 = (df_feat['f009_Engine-Major']==13000)
encode_idx_5 = (df_feat['f009_Engine-Major']==13804)
encode_idx_6 = (df_feat[vi_col_2]==125)
df_feat.loc[encode_idx_1, en_col] = 14901
df_feat.loc[encode_idx_2, en_col] = 14901
df_feat.loc[encode_idx_3, en_col] = 14901
df_feat.loc[encode_idx_4, en_col] = 14901
df_feat.loc[encode_idx_5, en_col] = 14901
df_feat.loc[encode_idx_6, en_col] = 14104

df_feat[df_feat[target].isnull()].loc[encode_idx][en_col].value_counts()
#========================================================================
feature = df_feat[en_col].values.astype('float32')
utils.to_pkl_gzip(obj=feature, path=f'../features/4_winner/{en_col}')

In [25]:
check_idx = (df_feat[en_col]==15200) & (df_feat['f012_AvSigVersion-Major']==126)
display(df_feat[df_feat[target].isnull()].loc[check_idx, :])
display(df_feat[~df_feat[target].isnull()].loc[check_idx, :])

,MachineIdentifier,HasDetections,country_group,f012_AvSigVersion-Major,f000_SmartScreen,f009_Engine-Major,f012_org-Engine-Major,f012_SmartScreen


,MachineIdentifier,HasDetections,country_group,f012_AvSigVersion-Major,f000_SmartScreen,f009_Engine-Major,f012_org-Engine-Major,f012_SmartScreen


In [ ]:
train = df_feat[~df_feat[target].isnull()]
test = df_feat[df_feat[target].isnull()]
#========================================================================
# Cleansing check 
en_col = 'f000_EngineVersion'
en_col_m = 'f009_Engine-Major'
en_col_m2 = 'f012_org-Engine-Major'
os_col = 'f000_Census_OSVersion'
vi_col = 'f000_AvSigVersion'
vi_col_2 = 'f012_AvSigVersion-Major'
app_col = 'f000_AppVersion'
sm_col = "f000_SmartScreen"
sm_col_2 = "f012_SmartScreen"
# train_cnt = train[train[sm_col].map(lambda x: 1 if str(x).count('quir') or str(x).count('xist') else 0 )==1].groupby([sm_col_2, en_col_m, en_col_m2, vi_col_2]).size()
# test_cnt = test[test[sm_col].map(lambda x: 1 if str(x).count('quir') or str(x).count('xist') else 0 )==1].groupby([sm_col_2, en_col_m, en_col_m2, vi_col_2]).size()
train_cnt = train.groupby([sm_col_2, en_col_m2, vi_col_2]).size()
test_cnt = test.groupby([sm_col_2, en_col_m2, vi_col_2]).size()
# train_cnt = train.groupby([sm_col_2, en_col_m2]).size()
# test_cnt = test.groupby([sm_col_2, en_col_m2]).size()
tmp = pd.concat([train_cnt, test_cnt], axis=1)
# tmp[tmp>1000].sort_values(by=1, ascending=False)
tmp[tmp[0].isnull()]
#========================================================================

In [ ]:
# tmp.reset_index(inplace=True)
# tmp[en_col_m2].unique()
# tmp[vi_col_2].unique()
engine_miner_set = [0., 12805., 12902., 13103., 13202., 13303., 13407., 13504.,
 13601., 13701., 13704., 13903., 14003., 14104., 14500., 14600.,
 14700., 14800., 14901., 15000., 15200., 14306., 14405., 14202.]
vi_miner_set = [124., 127., 122., 123., 125., 126.]
train.loc[(train[en_col_m2].isin(engine_miner_set)) & (train[vi_col_2].isin(vi_miner_set))]